In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

data_train = pd.read_csv('./data_set/Hotel-A-train.csv')
data_test = pd.read_csv('./data_set/Hotel-A-validation.csv')
data_out_or = pd.read_csv('./data_set/Hotel-A-test.csv')


In [26]:
def prepare(df,train):
    df=df.dropna()
    df=df.drop_duplicates()
    df = binAge(df)
    df = makeCats(df)
    df=df.dropna()
    df=df.drop_duplicates()
    if(train):
        y = df[["Reservation_Status"]]
        y = catStatus(y)
        return(df,y)
    else:
        ids = ids = df[["Reservation-id"]]
        return (df,ids)


def binAge(df):
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
    categories = pd.cut(df.Age, bins, labels=group_names)
    df[['Age_cat']] = categories.cat.codes
    return df

def binRoomRate(df):
    df.Room_Rate = df.Room_Rate.fillna(1)
    bins = (100,200,300)
    group_names = [1,2];
    categories = pd.cut(df.Room_Rate, bins, labels=group_names)
    df.Room_Rate = categories.cat.codes
    return df


def makeCats(x):
    x[["Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Discount_Rate"]]=x[["Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Discount_Rate"]].astype("category")
    cat_columns =["Gender_cat","Ethnicity_cat","Educational_Level_cat","Income_cat","Country_region_cat","Hotel_Type_cat","Meal_Type_cat","Visted_Previously_cat","Previous_Cancellations_cat","Deposit_type_cat","Booking_channel_cat","Required_Car_Parking_cat","Use_Promotion_cat","Discount_Rate_cat"]
    columns=["Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Discount_Rate"]
    for i in range(len(cat_columns)):
        x[cat_columns[i]]=x[columns[i]].cat.codes
    return x

def catStatus(df):
    cleanups ={"Reservation_Status":{"Check-In":1,"Canceled":2,"No-Show":3}}
    df = df.replace(cleanups)
    return df

def drop(df,columns):
    df = df.drop(columns = columns)
    return df
   
    
def out(df,columns,train):
    x,y = prepare(df,train)
    x = drop(x,columns)
    return (x,y)


In [27]:
columns_train=["Reservation-id","Reservation_Status","Expected_checkin","Expected_checkout","Booking_date","Age","Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Discount_Rate"]#,"Room_Rate"]
columns_out=["Reservation-id","Expected_checkin","Expected_checkout","Booking_date","Age","Gender","Ethnicity","Educational_Level","Income","Country_region","Hotel_Type","Meal_Type","Visted_Previously","Previous_Cancellations","Deposit_type","Booking_channel","Required_Car_Parking","Use_Promotion","Discount_Rate"]#,"Room_Rate"]




data_train_x,data_train_y = out(data_train,columns_train,True)
data_test_x,data_test_y =out(data_test,columns_train,True)
data_out,ids = out(data_out_or,columns_out,False)

#print(data_out.dtypes)

In [33]:
#print(data_train_x.sample(5))
print(data_out)

      Adults  Children  Babies  Room_Rate  Age_cat  Gender_cat  Ethnicity_cat  \
0          3         3       0        153        6           0              2   
1          2         1       0        210        6           0              2   
2          2         2       0        117        5           0              1   
3          1         3       2        107        7           0              3   
4          3         1       0        119        6           1              0   
...      ...       ...     ...        ...      ...         ...            ...   
4313       3         3       1        153        6           1              2   
4314       3         3       0        233        7           0              3   
4315       2         3       0        242        6           0              3   
4316       4         2       1        208        6           1              0   
4317       2         3       0        213        5           0              0   

      Educational_Level_cat

In [29]:
parameters = {'criterion': ['entropy', 'gini'],
              'splitter':['best','random'],
              'max_depth': [2, 3, 5, 10],
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1,5,8],
              'max_features': ['log2', 'sqrt','auto']   
             }
clf = tree.DecisionTreeClassifier()
#acc_scorer = make_scorer(accuracy_score)

#grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer)
#grid_obj = grid_obj.fit(data_train_x, data_train_y)

#clf = grid_obj.best_estimator_
clf = clf.fit(data_train_x, data_train_y)

In [31]:
pred = clf.predict(data_out)
ids[['Reservation_Status']]=pred
ids.to_csv('AGNI_C.csv',index=False)

ValueError: shape mismatch: value array of shape (4318,) could not be broadcast to indexing result of shape (1,4318)